## Titanic word2vec -> Raw data

In [59]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.models import Model

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import initializers
from keras.layers import regularizers
from keras.callbacks import EarlyStopping

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
 

from keras.optimizers import Adam, Adagrad, Adadelta

import keras.backend as K
from keras.utils.np_utils import to_categorical

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import chi2, SelectKBest

import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline


In [2]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(234)


In [38]:
train = pd.read_csv("../raw_data/titanic_train.csv")
train.shape

(891, 12)

### Prep for test submission

In [29]:
test = pd.read_csv("../raw_data/titanic_test.csv")

In [30]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [31]:
Pass = pd.DataFrame(test.PassengerId)

In [32]:
Pass.shape

(418, 1)

## Merge data for processing

In [39]:
df = train.append(test, ignore_index = True) #restart here if needed

In [40]:
junk = df.drop(['Survived'], axis = 1)

In [ ]:
y = df['Survived']

y.iloc[890, ]

y = y[:891, ]

y.tail(1)

y = pd.get_dummies(y, drop_first = True)

labels = np.asarray(y)

In [55]:
junk = junk.astype(str)
junk = junk.apply(lambda x: ' '.join(x), axis=1) 

In [58]:
print(junk.head(2))
junk.shape #should be a column vec

0    22.0 nan S 7.25 Braund, Mr. Owen Harris 0 1 3 ...
1    38.0 C85 C 71.2833 Cumings, Mrs. John Bradley ...
dtype: object


(1309,)

In [60]:
np.unique(y, return_counts = True) # pretty balanced

(array([0, 1], dtype=uint8), array([549, 342]))

In [61]:
tokenizer = Tokenizer(num_words = None)

In [62]:
tokenizer.fit_on_texts(junk)

In [63]:
sequences = tokenizer.texts_to_sequences(junk)

In [64]:
# compare, just note that the cleaned data is now a keras object, Junk is not cleaned
print(sequences[0:1])
print(junk[0:1])

[[35, 1, 2, 3, 11, 23, 654, 6, 655, 253, 1, 4, 7, 5, 4, 18, 13, 1189]]
0    22.0 nan S 7.25 Braund, Mr. Owen Harris 0 1 3 ...
dtype: object


In [65]:
word_index = tokenizer.word_index # type = dict

In [68]:
print('Found {} unique tokens.'.format(len(word_index)))

Found 4487 unique tokens.


In [69]:
data = pad_sequences(sequences, maxlen = None) #numpyarray of dim (samples, maxlen)

In [71]:
len(data[0])
len(data[1])

27

In [73]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (1309, 27)
Shape of label tensor: (891, 1)


In [74]:
rev_word_index = {v+3:k for k,v in word_index.items()}

In [75]:
rev_word_index[0] = 'padding_char'
rev_word_index[1] = 'start_char'
rev_word_index[2] = 'oov_char'
rev_word_index[3] = 'unk_char'

In [76]:
example_feat = ' '.join([rev_word_index[word] for word in data[3]])
example_feat #notice the zeros -> this is the padding. All lowercase, etc -> cleaned


'padding_char padding_char padding_char padding_char padding_char padding_char padding_char padding_char 20 start_char c85 unk_char 83 0 cumings 7 johnston 17599 6607 2833 heikkinen start_char 05 0 mr 0 c123'

In [77]:
data.shape

labels.shape

junk = data[:891, ]

(1309, 27)

In [172]:
X_test_f = data[891:]

In [173]:
X_test_f = X_test_f.reset_index(drop=True)

In [174]:
X_test_f.shape

(418, 27)

In [177]:
X_test_f = pd.DataFrame(X_test_f)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(junk, labels,
                                                    test_size=0.3)


In [83]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(623, 27)
(268, 27)
(268, 1)
(623, 1)


In [84]:
max_features = max([max(x) for x in X_train] + 
                   [max(x) for x in X_test]) + 1
max_features # number of features we have in the dictionary

3429

In [156]:
K.clear_session()

model = Sequential()
model.add(Embedding(max_features, 128))

model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [157]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=20,
          callbacks=[EarlyStopping(monitor='acc', patience=1)],
          validation_split=0.3)

Train on 436 samples, validate on 187 samples
Epoch 1/20
436/436 [==============================] - 3s 7ms/step - loss: 0.8255 - acc: 0.5459 - val_loss: 0.6145 - val_acc: 0.6684
Epoch 2/20
436/436 [==============================] - 2s 5ms/step - loss: 0.5612 - acc: 0.7248 - val_loss: 0.5513 - val_acc: 0.7273
Epoch 3/20
436/436 [==============================] - 2s 5ms/step - loss: 0.4998 - acc: 0.7752 - val_loss: 0.5330 - val_acc: 0.7380
Epoch 4/20
436/436 [==============================] - 2s 5ms/step - loss: 0.4150 - acc: 0.8234 - val_loss: 0.5649 - val_acc: 0.7059
Epoch 5/20
436/436 [==============================] - 2s 5ms/step - loss: 0.3479 - acc: 0.8670 - val_loss: 0.6027 - val_acc: 0.6684
Epoch 6/20
436/436 [==============================] - 2s 5ms/step - loss: 0.3023 - acc: 0.8853 - val_loss: 0.4330 - val_acc: 0.8128
Epoch 7/20
436/436 [==============================] - 2s 5ms/step - loss: 0.2452 - acc: 0.9128 - val_loss: 0.4455 - val_acc: 0.8182
Epoch 8/20
436/436 [==========

In [158]:
score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)


268/268 [==============================] - 0s 835us/step
Test score: 0.447656134171
Test accuracy: 0.817164178215


In [159]:
y_pred = model.predict(X_test)

y_pred_class = [ ]
for i in y_pred:
    if i >= 0.5:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)


confusion_matrix(y_test, y_pred_class)

array([[153,  17],
       [ 32,  66]])

In [160]:
print(classification_report(y_test, y_pred_class))

             precision    recall  f1-score   support

          0       0.83      0.90      0.86       170
          1       0.80      0.67      0.73        98

avg / total       0.82      0.82      0.81       268



In [161]:
model.save('titanic2_model.h5') 

In [162]:
model.evaluate(X_test, y_test)

268/268 [==============================] - 0s 728us/step


[0.44765613417127237, 0.8171641782148561]

In [180]:
X_test_f.shape

(418, 27)

In [181]:
y_pred = model.predict(X_test_f)

InvalidArgumentError: indices[0,22] = 3429 is not in [0, 3429)
	 [[Node: embedding_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_1/embeddings/read, embedding_1/Cast)]]

Caused by op 'embedding_1/Gather', defined at:
  File "/Users/jacobanderson/anaconda/envs/tfdeeplearning/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/jacobanderson/anaconda/envs/tfdeeplearning/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jacobanderson/anaconda/envs/tfdeeplearning/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jacobanderson/anaconda/envs/tfdeeplearning/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/jacobanderson/anaconda/envs/tfdeeplearning/lib/python3.5/site-packages/ipyker